## A training ground for sparse-coded autoencoders:

**I am inheriting thise materials from an archived private repo.**


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import  DataLoader
from torchvision import transforms, utils
import torch.nn.functional as F
#
from cv2 import imwrite
import os
#
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import dataTools as D

%precision 5

In [ ]:
database_name = 'CelebA'
############################
if database_name == 'CelebA':
    batch_size = 100
    root = 'path/to/data/CelebA/128_crop/'
    im_size = (128,128,3)
elif database == 'CYale':
    root = 'path/to/data/CYale'
    im_size = (168, 192,1)
#######################################
num_channel = im_size[2]
device = torch.device("cuda:1")

In [ ]:
fwd_transform = transforms.Compose([
    transforms.ToTensor(),  
                ])

In [ ]:

dataset = D.dataRead(root,im_size,transform=fwd_transform)
[dataset_train,dataset_test] = torch.utils.data.random_split(dataset,[int(len(dataset)* 0.8),1+int(len(dataset)* 0.2)])

dataloader_train = DataLoader(dataset_train, batch_size=batch_size,
                        shuffle=False, num_workers=1)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size,
                        shuffle=False, num_workers=1)

In [ ]:
# ########################################## Initializing the model ##############################
num_blocks = 6
num_filts = [40, 40, 40, 40, 40, 10]
scale_factor = [1, 2, 1, 2, 1, 2]
num_codes = 20
neck_dim = 512
k = 256
############
from models import Autoencoder
net = Autoencoder((im_size[2], im_size[0], im_size[1]), num_blocks, num_filts, scale_factor, num_codes, neck_dim, k).to(device)
net.parameters

In [ ]:
def loss_function(x_rec,x):
    loss_BCE = nn.BCELoss()
    #loss_MSE = nn.MSELoss()
    #return loss_L1_Charbonnier
    return loss_BCE(x_rec,x)
#############################################    
Loss_list_train = []
Loss_list_test = []
#############################################
optimizer = optim.Adam(net.parameters(), lr=0.0099,weight_decay=0)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 'min', verbose=True, factor=0.99, min_lr=0.000001,patience=100)

In [ ]:
# ########################################## Training a new model? ##############################
num_epoch = 200
for i_epoch in range(num_epoch):
    loss_train = 0.0
    print('---------------- epoch = ', i_epoch + 1, '/',num_epoch, ' ----------')
    for i_batch, inputs_train in enumerate(dataloader_train):
        # print('batch = ',i_batch,end='  ')

        inputs_train = inputs_train['image'].to(device) 
        outputs_train,code_train = net(inputs_train)
        outputs_train.sigmoid_()
        
        
        optimizer.zero_grad()
        loss = loss_function(outputs_train,inputs_train)
        loss.backward()  
        optimizer.step()
        scheduler.step(loss)
        
        loss_train += loss.item()
        if i_epoch < 2:
             print('loss = ', loss.item(), end='  ') 
    print('Avg train loss = ',loss_train/len(dataloader_train))
    Loss_list_train.append(loss_train/len(dataloader_train))
    with torch.no_grad():
        loss_test = 0.0
        for _, inputs_test in enumerate(dataloader_test):
            #name_test = inputs_test['name']
            inputs_test = inputs_test['image'].to(device) 
            outputs_test,code_test = net(inputs_test)
            outputs_test.sigmoid_()
            
            loss_test += loss_function(outputs_test,inputs_test).item()
        print('Avg test  loss = ',loss_test/len(dataloader_test))  
        Loss_list_test.append(loss_test/len(dataloader_test))

In [ ]:
plt.plot(Loss_list_train)
plt.plot(Loss_list_test)

In [ ]:
# ## To run the network (again) on the test set?
# ### To-Do: No need to iterate on the entire set. Just take a mini-batch.
# with torch.no_grad():
#     for _, inputs_test in enumerate(dataloader_test):
#         inputs_test = inputs_test['image'].to(device) 
#         outputs_test,code_test = net(inputs_test)

In [ ]:
########################################## Saving the model? ##############################
def list2str(L):
    s = ''
    for i,l in enumerate(L):
        s += str(l)
        if i < len(L) - 1:
            s += '-'
    return s
#########################
model_name = database_name + \
            '_filts' + list2str(num_filts)+ \
            '_scale' + list2str(scale_factor) +\
            '_codes' + str(num_codes) +\
            '_dim' + str(neck_dim) +\
            '_k' + str(k) + \
            '.pth'

print(model_name)
date = '2019-10-14'
#os.mkdir(os.path.join('./state_dict/', date))
##########################
torch.save(net.state_dict(),os.path.join('./state_dict/', date, model_name))

In [ ]:
##################### Saving the names of test images, for future run:
names_test = []
for _, inputs_test in enumerate(dataloader_test): 
    names_test.extend(inputs_test['name'])
##############################################
f = open(os.path.join('./state_dict/', date, 'test_set_' + model_name[:-4] + '.txt'),"x")
f.write("\n".join(names_test))
f.close() 

In [ ]:
with torch.no_grad():
    plt.plot(inputs_train[0,:].view(-1,1).cpu().numpy()[0:7000])
    plt.plot(outputs_train[0,:].view(-1,1).cpu().numpy()[0:7000])

In [ ]:
plt.plot(inputs_test[10,:].view(-1,1).cpu().numpy()[0:1000])
plt.plot(outputs_test[10,:].view(-1,1).cpu().numpy()[0:1000])

In [ ]:
plt.imshow(inputs_test[0,:,:,:].squeeze(0).transpose(0,2).cpu().numpy())

In [ ]:
with torch.no_grad():
    plt.imshow(outputs_test[0,:,:,:].squeeze(0).transpose(0,2).cpu().numpy())

In [ ]:
with torch.no_grad():
    code_tilde = torch.clone(code_test)
    i_s = 0
    i_e = 19
    i_item = 19
    # code_tilde =  2 * torch.ceil(code_tilde/2)
    code_tilde[0:12,i_s:i_e,:] += code_tilde[i_item:i_item + 1,i_s:i_e,:]# + code_tilde[18:19 ,i_s:i_e,:]
    #code_tilde[0:12,i_s:i_e,:] = 0
    out_tilde = net.decoder(code_tilde).sigmoid_()
    plt.imshow(out_tilde[0,:,:,:].squeeze(0).transpose(0,2).cpu().numpy())
    

In [ ]:
print(code_test.shape)
#plt.plot(code_test[1,0,:].cpu().numpy())
plt.plot(code_test[10,0,:].cpu().numpy())
plt.plot(code_tilde[10,0,:].cpu().numpy())

In [ ]:
print(torch.nonzero(code_test).shape)
print(torch.nonzero(code_tilde).shape)

In [ ]:
plt.plot(outputs_test[0,:,:,:].view(-1,1).cpu().numpy()[7000:8000])
plt.plot(out_tilde[0,:,:,:].view(-1,1).cpu().numpy()[7000:8000])

In [ ]:
#### Ssaving some samples:
utils.save_image(inputs_test[0:12,:,:,:],'samples/inputs.png',nrow=6)
utils.save_image(outputs_test[0:12,:,:,:],'samples/outputs.png',nrow=6)
utils.save_image(out_tilde[0:12,:,:,:],'samples/tildes.png',nrow=6)

In [ ]:
torch.norm(inputs_test - outputs_test).pow(2)/torch.norm(inputs_test).pow(2)